1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있음.
    - 문서가 길면 답변 생성이 오래걸림
3. 임베딩 하여 백터 데이터에 저장 
4. 질문이 있을 때, 백터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM 에 질문과 같이 전달.

## 1. 문서의 내용을 읽는다.

In [2]:
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("./tax.docx")
document = loader.load()

len(document)

1

In [3]:
len(document[0].page_content)

238228

## 2. 문서를 쪼갠다.

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=20
)

document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

168

## 3. 임베딩 하여 백터 데이터에 저장 

In [5]:
from langchain_upstage import UpstageEmbeddings
from dotenv import load_dotenv

load_dotenv()
embeddings = UpstageEmbeddings(model="solar-embedding-1-large") 

In [ ]:
# %pip install langchain-pinecone

from dotenv import load_dotenv
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore


load_dotenv()
index_name = 'tax-index'

# Set your Pinecone API key here if not already set
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embeddings, index_name=index_name)

ValueError: Index 'tax-index' not found in your Pinecone project. Did you mean one of the following indexes: tax-index-markdown

## 4. 질문이 있을 때, 백터 데이터베이스에 유사도 검색

In [1]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retriever = database.similarity_search(query, k=3)
print(retriever)

NameError: name 'database' is not defined

## 5. 유사도 검색으로 가져온 문서를 LLM 에 질문과 같이 전달.

In [29]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [ ]:
# DB 에 as_retriever 를 사용하여 연결.
# as_retriever 는 chroma 뿐만 아니라 다양한 데이터베이스에서 사용할 수 있다.
# retrieverQA 를 활용하여 답변 고도화하기
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

ai_message = qa_chain.invoke(query)
print(ai_message)

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?', 'result': '연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 우선 근로소득공제를 적용해야 합니다. 2024년 기준으로 총급여액 5천만원인 경우 근로소득공제액은 1천2백만원으로 공제 후 근로소득금액은 3천8백만원이 됩니다. 이후 기본공제, 표준세액공제, 근로소득세액공제 등을 적용하여 계산하면, 대략 400만원 초반대의 소득세가 부과됩니다. 그러나 이는 대략적인 계산이며, 실제 소득세는 개인의 상황에 따라 다양한 공제 항목이 적용될 수 있으므로 정확한 금액은 확인할 수 없습니다.'}


 '연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 우선 근로소득공제를 적용해야 합니다. 2024년 기준으로 
 총급여액 5천만원인 경우 근로소득공제액은 1천2백만원으로 공제 후 근로소득금액은 3천8백만원이 됩니다. 이후 기본공제, 표준세액공제, 근로소득세액공제 등을 적용하여 계산하면, 대략 400만원 초반대의 소득세가 부과됩니다. 그러나 이는 대략적인 계산이며, 실제 소득세는 개인의 상황에 따라 다양한 공제 항목이 적용될 수 있으므로 정확한 금액은 확인할 수 없습니다.'